In [1]:
!pip3 install nirvana_api vh

Looking in indexes: https://pypi.yandex-team.ru/simple/
     - 32.6 kB 85.1 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
     - 1.5 MB 30.5 MB/s 0:00:000m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 96.5 MB/s eta 0:00:00
     - 1.5 MB 29.1 MB/s 0:00:000m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.0/334.0 kB 26.4 MB/s eta 0:00:00
  Created wheel for nirvana_api: filename=nirvana_api-2.5.0.post13470917-py3-none-any.whl size=39751 sha256=ca4ac94913a84994a6bb9134a6dfa4d775f5a2fccc3c6c7a356d43fa902f377e
  Stored in directory: /home/notantonvoron/.cache/pip/wheels/49/31/99/8e77f5c23758471034828613dae939b392911c5aadbdb8454b
Successfully built nirvana_api


In [2]:
%reload_ext autoreload
%autoreload 2
%env REQUESTS_CA_BUNDLE=/usr/local/share/ca-certificates/YandexInternalRootCA.crt

env: REQUESTS_CA_BUNDLE=/usr/local/share/ca-certificates/YandexInternalRootCA.crt


In [8]:
import nirvana_api
import vh


def run_graph(
    workflow_guid,
    comment,
    priority,
    nirvana_token,
    ssh_secret,
    code_to_run,
    gpu_count=1,
):
    with vh.Graph() as g:
        NIRVANA_API = nirvana_api.NirvanaApi(nirvana_token, ssl_verify=False)

        clone = vh.op(id="a918d982-9a42-4519-884b-f8759e41dc08")

        clone_res = clone(
            repo="git@github.com:RealAntonVoronov/metaicl_reproduction.git",
            folder_name="MetaICL",
            commit_or_branch="fixes_and_logging",
            key=ssh_secret,
        )
        
        data = vh.data(id='a3a83880-9b4e-4dfe-8f6e-f363fe86451a', quota='yr-other')
        volume = vh.data(id='20eeb072-1690-4b13-86fb-52986a0253e7', quota='yr-other')
            
        pydl_op = vh.op(id='6f51349b-2206-41a4-ab2b-e5767502d6bf')

        pydl_op(
            _inputs=dict(
                volume=volume, script=clone_res, data=data
            ),
            command=code_to_run,
            gpu_count=gpu_count,
            gpu_type='CUDA_8_0',
            gpu_max_ram=75000,
            max_disk=512000,
            max_ram=250000,
            cpu_guarantee=12000,
            ttl=2880,
        )

    keeper = vh.run(
        graph=g,
        oauth_token=nirvana_token,
        quota='yr-other',
        workflow_guid=workflow_guid,
        description=comment,
        start=False,
    )
    info = keeper.get_workflow_info()
    NIRVANA_API.edit_workflow(
        workflow_id=info.workflow_id,
        workflow_instance_id=info.workflow_instance_id,
        execution_params={'workflowPriority': priority},
    )
    NIRVANA_API.start_workflow(
        workflow_id=info.workflow_id, workflow_instance_id=info.workflow_instance_id
    )

In [9]:
euler = "http://euler.sas.yp-c.yandex.net:6953"
WORKFLOW_ID = "eaa6aef4-537a-4c55-90e0-726cb70a84ed"
NIRVANA_TOKEN = 'y1_AQAD-qJSL8cVAAACjwAAAAAALtHJ7r7d6v_eQ_qBqVSetJvGYbeoWpQ'

SSH_SECRET = "eccdsa_notantonkey"

In [7]:
CODE_PRELUDE = f"""set -euxo pipefail
export {{http,https,ftp}}_proxy="http://euler.sas.yp-c.yandex.net:6953"

export MAMBA_ROOT_PREFIX="/micromamba"
export PYTHON_MAIN_ENV=$MAMBA_ROOT_PREFIX/envs/main
export PATH=$PATH:$PYTHON_MAIN_ENV/bin
export PYTHON_CMD=$PYTHON_MAIN_ENV/bin/python
export YT_PROXY=hahn.yt.yandex.net
export WANDB_API_KEY="49a175fa5aba376a381ff86821b8f183b3fc8fa3"

$PYTHON_CMD -m pip install transformers wandb bitsandbytes
export TRANSFORMERS_CACHE=$INPUT_PATH

cd $SOURCE_CODE_PATH/MetaICL
"""
n_gpu = 2
batch_size = 2
lr = 1e-05
command = ("$PYTHON_CMD train.py --n_gpu 8 --use_demonstrations --task hr_to_lr --k 16384 --test_k 16 --seed 100 "
           f"--method channel --train_seed 1 --batch_size {batch_size} --lr {lr} --fp16 "
           "--out_dir $SNAPSHOT_PATH/checkpoints/channel-metaicl-llama_2_7b/hr_to_lr "
           "--gpt2 meta-llama/Llama-2-7b-hf --tensorize_dir $INPUT_PATH/tensorized_llama_fix_eos/ --optimization 8bit-adam"
          )

comment = f"metaicl_train_llama2-7b_bs-{batch_size}_lr-{lr}_ngpu_{n_gpu}"

run_graph(
    WORKFLOW_ID,
    comment=comment,
    priority="normal",
    nirvana_token=NIRVANA_TOKEN,
    ssh_secret=SSH_SECRET,
    code_to_run=CODE_PRELUDE + command,
    gpu_count=n_gpu,
)

NameError: name 'data_id' is not defined